In [1]:
import openpyxl
import xlrd

In [2]:
change_log_rev_coloum = 2  # The colomn of `Rev#`` of `Change Log`.

In [3]:
change_log_what_changed_coloum = 6  # The colomn of `What Changed` of `Change Log`.

In [4]:
change_log_who_coloum = 4  # The colomn of `Who` of `Change Log`.

In [5]:
job_start_coloum = 7  # The colomn where the `job` attribute starts.

In [6]:
part_name_coloum = 2  # The colomn of `part name`

In [7]:
part_number_coloum = 3  # The colomn of `part number`

In [8]:
tool_count_coloum = [5, 6]  # The colomn of `tool count`

In [9]:
tool_coloum = 4  # The colomn of `tool`

In [10]:
# To handle merged cells, use this function for all read operation. 
def get_cell_value(sheet, row_index, col_index):
    cell_value = sheet.cell(row_index, col_index).value
    for r in sheet.merged_cells.ranges:
        rlow = r.min_row
        rhigh = r.max_row
        clow = r.min_col
        chigh = r.max_col
        if (row_index >= rlow and row_index <= rhigh and col_index >= clow and col_index <= chigh):
            cell_value = sheet.cell(rlow, clow).value
            break
            
    if cell_value is None:
        cell_value = ''
    return cell_value

In [11]:
def get_cell_note(sheet, row_index, col_index):
    note = None
    if sheet.cell(row_index, col_index).comment:
        note = sheet.cell(row_index, col_index).comment.text
    if note is None:
        for r in sheet.merged_cells.ranges:
            rlow = r.min_row
            rhigh = r.max_row
            clow = r.min_col
            chigh = r.max_col
            if (row_index >= rlow and row_index <= rhigh and col_index >= clow and col_index <= chigh):
                if sheet.cell(rlow, clow).comment:
                    note = sheet.cell(rlow, clow).comment.text
                    break
    if note is None:
        note = ''
    return note

In [12]:
def fill(r_row, w_row, r_sheet, r_header, w_sheet, w_header, section_index, section_start_row, section_end_row):

    for c in range(job_start_coloum, len(r_header) + 1):
        # If maker is empty, skip this row
        maker = get_cell_value(r_sheet, r_row, c)
        if maker == '':
            continue
        
        # If the first three columns are empty, skip this row
        part_name = get_cell_value(r_sheet, r_row, part_name_coloum)
        part_number = get_cell_value(r_sheet, r_row, part_number_coloum)
        tool = get_cell_value(r_sheet, r_row, tool_coloum)
        if part_name == '' and part_number == '' and tool == '':
            break
 
        job_family = get_cell_value(r_sheet, section_start_row[section_index]-1, c)
        w_sheet.cell(w_row, w_header.index('job_family') + 1).value = job_family

        # change tool if job_family in 'Blank Die' or 'Fixtures' or 'Plates' or 'Jigs'
        if 'Blank Die' in job_family:
            w_sheet.cell(w_row, w_header.index('tool') + 1).value = 'Blanker'
        elif 'Fixtures' in job_family:
            w_sheet.cell(w_row, w_header.index('tool') + 1).value = 'Fixture'
        elif 'Plates'  in job_family:
            w_sheet.cell(w_row, w_header.index('tool') + 1).value = tool + '_Plates'
        elif 'Jigs' in job_family:
            w_sheet.cell(w_row, w_header.index('tool') + 1).value = tool + '_Jigs'
        else:
            w_sheet.cell(w_row, w_header.index('tool') + 1).value = tool
        
        # Fill in fixed values.
        w_sheet.cell(w_row, w_header.index('vehicle_my') + 1).value = vehicle_my
        w_sheet.cell(w_row, w_header.index('vehicle_name') + 1).value = vehicle_name
        w_sheet.cell(w_row, w_header.index('vehicle_prog_type') + 1).value = vehicle_prog_type
        w_sheet.cell(w_row, w_header.index('vehicle_seq') + 1).value = vehicle_seq
        w_sheet.cell(w_row, w_header.index('vehicle_dev_source') + 1).value = vehicle_dev_source
        w_sheet.cell(w_row, w_header.index('plant') + 1).value = plant
        w_sheet.cell(w_row, w_header.index('rev') + 1).value = rev
        w_sheet.cell(w_row, w_header.index('rev_date') + 1).value = rev_date
        w_sheet.cell(w_row, w_header.index('rev_assoc') + 1).value = rev_assoc
        w_sheet.cell(w_row, w_header.index('rev') + 1).value = rev
        w_sheet.cell(w_row, w_header.index('rev_note') + 1).value = rev_note

        # Fill tool_type. Different sections have different ways of judging.
        if section_index == 0:
            tool_type = 'ST Die'
        elif section_index == 1:
            tool_type = 'WE Hem Die'
        elif section_index == 2:
            if tool == 'Cavity' or tool == 'Core':
                tool_type = 'Po mold'
            elif tool == 'tooling':
                tool_type = 'PO Tool'
            else:
                tool_type = ''
        else:
            # There are only three sections in each sheet.
            raise NotImplementedError
        w_sheet.cell(w_row, w_header.index('tool_type') + 1).value = tool_type

        # fill tool_count
        tool_count_a = get_cell_value(r_sheet, r_row, tool_count_coloum[0])
        tool_count_b = get_cell_value(r_sheet, r_row, tool_count_coloum[1])
        tool_count_a = 0 if tool_count_a == '' else tool_count_a
        tool_count_b = 0 if tool_count_b == '' else tool_count_b
        # change tool_count if job_family in 'Blank Die' or 'Fixtures' or 'Plates' or 'Jigs'
        if 'Blank Die' in job_family:
            tool_count = tool_count_b
        elif 'Fixtures' in job_family or 'Plates'  in job_family or 'Jigs' in job_family:
            tool_count = tool_count_a
        else:
            tool_count = max(tool_count_a, tool_count_b)
        w_sheet.cell(w_row, w_header.index('tool_count') + 1).value = tool_count


        job = get_cell_value(r_sheet, section_start_row[section_index], c)
        w_sheet.cell(w_row, w_header.index('job') + 1).value = job
        
        job_descripition = get_cell_note(r_sheet, section_start_row[section_index], c)
        w_sheet.cell(w_row, w_header.index('job_descripition') + 1).value = job_descripition

        # get maker comment and judge part name and number
        maker = get_cell_value(r_sheet, r_row, c)
        w_sheet.cell(w_row, w_header.index('maker') + 1).value = maker
        maker_comment = get_cell_note(r_sheet, r_row, c)

        if "common" in maker_comment:
            new_name = maker_comment.split(':')[1]
            start_row = section_start_row[section_index]
            end_row = section_end_row[section_index]
            for i in range(start_row,end_row):
                current_part_name = get_cell_value(r_sheet,i,part_name_coloum)
                if current_part_name in new_name:
                    new_part_number = get_cell_value(r_sheet,i,part_number_coloum)
                    part_name = part_name + " & " + new_name
                    part_number = str(part_number) + " & " + str(new_part_number)
                    break
        w_sheet.cell(w_row, w_header.index('part_name') + 1).value = part_name
        w_sheet.cell(w_row, w_header.index('part_number') + 1).value = part_number
        w_sheet.cell(w_row, w_header.index('maker_comment') + 1).value = maker_comment
        w_row += 1
    return w_row

In [13]:
rb = openpyxl.load_workbook("Copy of NM Responsibility (ST & PO) Rolling.xlsx") # Input the name of the table to read

C:\Users\Administrator\anaconda3\lib\site-packages\openpyxl\reader\excel.py:240: UserWarning: Cell '22-CIVIC-FMC-HMIN-4DR':I15 is part of a merged range but has a comment which will be removed because merged cells cannot contain any data.
  warnings.warn(comment_warning.format(ws.title, c.coordinate))


In [14]:
w_sheet = rb["sample"]

In [15]:
w_header = []

In [16]:
for cell in w_sheet['1']:
    w_header.append(cell.value)

In [17]:
# find sheet_name from sheet administrator
w_sheet_names = rb["Administration"]

In [18]:
sheet_name = []

In [19]:
for i in range(2, w_sheet_names.max_row + 1):
    s_name = get_cell_value(w_sheet_names, i, 2)
    s_name2 = w_sheet_names.cell(i, 2).value
    if s_name != '':
        sheet_name.append(s_name)

In [20]:
w_row = 2

In [21]:
for sheet_index, sn in enumerate(sheet_name):
    print("Now process sheet {}: {}".format(sheet_index, sn))
    r_sheet = rb[sn]

    # get the section start-end row of this sheet
    section_start_row = [9]
    section_end_row = []

    # Search for the beginning and end of each section based on the specified field
    for i in range(1, r_sheet.max_row + 1):
        cell_value = get_cell_value(r_sheet, i, 2)
        if "Stamping Total" in cell_value:
            section_start_row.append(i + 2)
            section_end_row.append(i - 1)
            continue
        if "Hemming Total" in cell_value:
            section_start_row.append(i + 2)
            section_end_row.append(i - 1)
            continue
        if "PO Total" in cell_value:
            section_end_row.append(i - 1)
            break


    # get seq and rev info column from every sheet
    vehicle_seq_pos_l = 0
    vehicle_dev_source_pos_l = 0
    rev_pos_l = 0
    rev_date_pos_l = 0
    rev_assoc_pos_l = 0
    for i in range(1, r_sheet.max_column + 1):
        cell_value = get_cell_value(r_sheet, 4, i)
        if "Seq #" in cell_value:
            vehicle_seq_pos_l = i
            continue
        if "development Location" in cell_value:
            vehicle_dev_source_pos_l = i
            continue
        if "Rev #" in cell_value:
            rev_pos_l = i
            continue
        if "Rev Date" in cell_value:
            rev_date_pos_l = i
            continue
        if "REV associate" in cell_value:
            rev_assoc_pos_l = i
            continue

    vehicle_seq_pos = (6, vehicle_seq_pos_l)  # The (row, column) of `seq#`. Notice in openpyxl that the index starts at 1.
    vehicle_dev_source_pos = (6, vehicle_dev_source_pos_l)  # The (row, column) of `Development Location`.
    rev_pos = (6, rev_pos_l)  # The (row, column) of `Rev#``.
    rev_date_pos = (6, rev_date_pos_l)  # The (row, column) of `Rev Date`.
    rev_assoc_pos = (6, rev_assoc_pos_l)  # The (row, column) of `REV associate`.


    # get values of some fields based on the name and attribution of the sheet
    tokens = sn.split('-')
    vehicle_my = tokens[0]
    vehicle_name = tokens[1] + '-' + tokens[-1]
    vehicle_prog_type = tokens[2]
    vehicle_seq = get_cell_value(r_sheet, vehicle_seq_pos[0], vehicle_seq_pos[1]).split('_')[-1]
    vehicle_dev_source = get_cell_value(r_sheet, vehicle_dev_source_pos[0], vehicle_dev_source_pos[1]).split('-')[0]
    plant = tokens[-2]
    rev = int(get_cell_value(r_sheet, rev_pos[0], rev_pos[1]))
    date = get_cell_value(r_sheet, rev_date_pos[0], rev_date_pos[1])
    rev_date = date.__format__('%Y/%m/%d')
    rev_assoc = get_cell_value(r_sheet, rev_assoc_pos[0], rev_assoc_pos[1])
    rev_note = None
    # Locate to the newest 'Change Log'
    for i in range(1, r_sheet.max_row + 1):
        if get_cell_value(r_sheet, i, change_log_rev_coloum) == rev:
            rev_note = get_cell_value(r_sheet, i, change_log_what_changed_coloum)

    # Scan the sheet section by section.
    for section_index in range(len(section_start_row)):
        # Get the `job` field for each section.
        r_header = []
        for cell in r_sheet[str(section_start_row[section_index])]:
            r_header.append(cell.value)
        for idx in range(len(r_header)-1, 0, -1):
            if r_header[idx] == '' or r_header[idx] is None:
                r_header.pop()
            else:
                break
        # Scan the section row by row.
        for current_row in range(section_start_row[section_index] + 1, section_end_row[section_index]):
            w_row = fill(current_row, w_row, r_sheet, r_header, w_sheet, w_header, section_index, section_start_row, section_end_row)

Now process sheet 0: 23-ACCORD-FMC-MAP-4DR
Now process sheet 1: 22-CIVIC-FMC-HMIN-4DR
Now process sheet 2: 22-CIVIC-FMC-HMIN-5DR


In [22]:
result_file = "result.xlsx"

In [23]:
rb.save(result_file)

In [24]:
print("Please view the results you want in file {}: sheet sample!".format(result_file))

Please view the results you want in file result.xlsx: sheet sample!
